## Searching for bouts for a day of alsa recording

In [21]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-03-25 01:19:01,776 root         INFO     Running on pouli
2022-03-25 01:19:01,776 root         INFO     Running on pouli


In [22]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [23]:
reload(et)
sess_par = {'bird': 's_b1267_22',
           'sess': '2022-03-24'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [24]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1267_22',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa',
 'kwik': '/experiment/s_b1267_22/alsa/kwik/2022-03-24',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1267_22/2022-03-24/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-24/alsa',
 'tmp': '/experiment/tmp/tmp',
 'msort': '/experiment/tmp/s_b1267_22/alsa/msort/2022-03-24',
 'ksort': '/experiment/tmp/s_b1267_22/alsa/ksort/2022-03-24/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-24/alsa/'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [25]:
reload(sb)
hparams, bpd = sb.read_session_bouts(sess_par['bird'], sess_par['sess'], recording_software='alsa', curated=False)

In [26]:
bpd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1017030,1022560,48817440,49082880,"[0.8933419972796475, 3.8511135729521118, 4.651...",0.894015,93.515268,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,5530,"[[1017030, 1017160], [1017715, 1017770], [1018...",17,"[3, 10, 21, 140, 199, 206, 215, 226, 236, 243,...",58,95.344828,"[-3, -2, -2, -2, -3, -4, -4, -4, -4, -5, -6, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
1,650030,657875,31201440,31578000,"[0.9234548419776489, 2.164746983418609, 2.4988...",3.260104,97.669336,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,7845,"[[650030, 650390], [652345, 652440], [653680, ...",10,"[12, 25, 32, 44, 52, 60, 70, 471, 480, 730, 73...",56,140.089286,"[-3, -4, -5, -5, -6, -6, -6, -5, -3, -3, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
2,665605,678435,31949040,32564880,"[8.65376102719656, 18.660414083504023, 22.9122...",3.260104,110.935921,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,12830,"[[665605, 665665], [665780, 665925], [665945, ...",54,"[3, 53, 61, 69, 82, 148, 193, 202, 246, 257, 2...",159,80.691824,"[1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 2, 1, 2, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
3,1278920,1290210,61388160,61930080,"[1.9607897985679723, 1.5181693614365859, 3.202...",3.260104,108.519652,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,11290,"[[1278920, 1278980], [1280730, 1280820], [1280...",23,"[5, 367, 376, 406, 413, 422, 432, 443, 450, 46...",80,141.125000,"[-24, -24, -23, -24, -23, -22, -22, -21, -19, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
4,13310,32670,638880,1568160,"[1.5719644835004982, 1.9296255524218635, 2.629...",5.824891,94.551948,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,19360,"[[13310, 13555], [13565, 13665], [14325, 14910...",60,"[8, 18, 28, 45, 54, 62, 71, 209, 219, 226, 237...",194,99.793814,"[4, 5, 4, 3, 4, 5, 3, 2, 3, 4, 7, 6, 5, 6, 7, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True


In [27]:
bpd.index.size

75

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

## inspect the bouts and curate them

#### visualize one bout

In [28]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [29]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

#viz_one_bout(bpd.iloc[1])

## use it in a widget


In [30]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [31]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        self.audio = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        #self.audio = Audio(data=np.zeros(100), rate=self.s_f)
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                  widgets.VBox([self.m_pick])]
                                  )
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
        #self.audio = Audio(data=self.x.flatten(), rate=self.s_f)
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [35]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [36]:
viz_bout.bouts_pd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1017030,1022560,48817440,49082880,"[0.8933419972796475, 3.8511135729521118, 4.651...",0.894015,93.515268,True,/mnt/sphere/speech_bci/raw_data/s_b1267_22/202...,5530,"[[1017030, 1017160], [1017715, 1017770], [1018...",17,"[3, 10, 21, 140, 199, 206, 215, 226, 236, 243,...",58,95.344828,"[-3, -2, -2, -2, -3, -4, -4, -4, -4, -5, -6, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False


In [37]:
np.where((viz_bout.bouts_pd['bout_check']==True) & (viz_bout.bouts_pd['confusing']==False))[0].size

66

### save it

In [40]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

os.makedirs(bouts_folder, exist_ok=True, mode=0o777)

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2022-03-25 11:07:30,022 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-24/alsa/bouts_ceciestunepipe/bout_checked.pickle
2022-03-25 11:07:30,022 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1267_22/2022-03-24/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [ ]:
viz_bout.bouts_pd['file'][0]

In [41]:
viz_bout.bouts_pd[viz_bout.bouts_pd['bout_check']==True]['file'].values

array(['/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/07-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/alsa/08-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1267_22/2022-03-24/als

In [ ]:
viz_bout.bouts_pd.head(5)

### attempts at audio playing within the widget

In [ ]:
a = widgets.Audio()
a.value = bytes(viz_bout.x)